In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
local_path = "/your/path/here"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/input')

['complaints-merged-2015',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-oct',
 'complaints-cpd-2016-nov',
 'complaints-ipra-2016-apr',
 'shootings-ipra-may2016',
 'shootings-cpd-feb2016',
 'complaints-cpd-2016-dec',
 'TRRs']

In [5]:
in_path = path + '/input/complaints-ipra-2016-apr/'
out_path = local_path + '/output/complaints-ipra-2016-apr/'

In [6]:
location_code = db.download_file(path+'/doc/','Location_Code_Dictionary.csv')
location_code.dropna(how='all', inplace=True)
location_code['Location_Code']=location_code['Location_Code'].astype(int).astype(str)

/Data/Import/doc/location_code_dictionary.csv


In [7]:
location_code.head()

,Location_Code,Location_Value
0,1,Food Sales/Restaurant
1,2,Tavern/Liquor Store
2,3,Other Business Establishment
3,4,Police Building
4,5,Lockup Facility


## Apr 2016 Data

### Report 1

In [8]:
in_path_apr_2016_report = in_path
out_path_apr_2016_report = out_path

files = [x for x in db.list_files(in_path_apr_2016_report) if '.xls' in x]
files

['All complaints during 2012.xls',
 'All complaints during 2013.xls',
 'All complaints during 2014.xls',
 'All complaints during 2015.xls',
 'All complaints during 2016YTD.xls',
 "IPRA Investigators' Info.xls",
 'Shootings Victim Data.xls']

In [ ]:
comp_files = [file for file in files if 'complaints' in file]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()
for file in comp_files:
    ##df = pd.read_excel(in_path_apr_2016_report + file)
    df = db.download_file(in_path_apr_2016_report,file)
    ## Making Sure Every File contains date the file was created and the foia that created it
    #col_list = df.columns.tolist()
    #Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
    #col_list = [x for x in col_list if isinstance(x, datetime.datetime)==False]
    #FOIA_Request = [x for x in col_list if 'FOIA' in x][0]
    
    df.dropna(how='all', inplace=True)
    #print(df.head())

    df.columns = ["CRID","Beat","Street","Address","Location_Value","Incident_Time_Start","Incident_Time_End",
                  "Notification_Date","Closed_Date","Investigator_Last_Name","Investigator_First_Name",
                  "Accused_First_Name","Accused_Last_Name","Accused_Star","Accused_Unit","Accused_Detail",
                  "Accused_Appointment_Date","Accused_Rank","Initial_Category_Code","Initial_Category_Description",
                  "Current_Category_Code","Current_Category_Description","Current_Status","Finding_Code","Disciple_Code",
                  "Discipline_Description","Involved_Officer_Last_Name","Involved_Officer_First_Name",
                  "Involved_Officer_Unit","Involved_Officer_Detail","Involved_Officer_Rank",
                  "Involved_Officer_Age","Involved_Officer_Sex","Involved_Officer_Race"]


    ## Covert Address Columns into single column
    df["Address"] = df['Address'].astype(str)
    df["Street"] = df["Street"].astype(str)
    df["Address"] = df["Address"].str.replace("nan","")
    df["Address"] = df["Address"].str.replace(".0","")
    df["Street"] = df["Street"].str.replace("nan","")

    df["Address_of_Incident"] = df[["Address","Street"]].apply(lambda x: ' '.join(x), axis=1)

    df = df[["CRID","Beat","Address","Street","Address_of_Incident","Location_Value","Incident_Time_Start",
             "Incident_Time_End","Notification_Date","Closed_Date","Investigator_Last_Name","Investigator_First_Name",
             "Accused_First_Name","Accused_Last_Name","Accused_Star","Accused_Unit","Accused_Detail",
             "Accused_Appointment_Date","Accused_Rank","Initial_Category_Code","Initial_Category_Description",
             "Current_Category_Code","Current_Category_Description","Current_Status","Finding_Code","Disciple_Code",
             "Discipline_Description","Involved_Officer_Last_Name","Involved_Officer_First_Name",
             "Involved_Officer_Unit","Involved_Officer_Detail","Involved_Officer_Rank",
             "Involved_Officer_Age","Involved_Officer_Sex","Involved_Officer_Race"]]

    
    ## Adding File Metadata
    ##df["FOIA_Request_Number"]=FOIA_Request
    try:
        df["Report_Produced_Date"]=Report_Produced_Date.date()
    except:
        df["Report_Produced_Date"]=''    

    final_df = final_df.append(df)
    final_df.reset_index(drop=True,inplace=True)
    metadata_df = metadata_df.append(metadata_dataset(df,file))
    metadata_df.reset_index(drop=True,inplace=True)

/Data/Import/input/complaints-ipra-2016-apr/all complaints during 2012.xls


In [9]:
final_df.head()

NameError: name 'final_df' is not defined

In [12]:
saving_files = "All_complaints_during_2012-2016YTD"

final_df.to_csv(out_path_apr_2016_report+saving_files+".csv",index=False)
final_df.to_excel(out_path_apr_2016_report+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path_apr_2016_report+saving_files+"_metadata.csv",index=False)

### Report 2

In [13]:
## FILE SEEMS TO CONTAIN 2 SEPARATE FILES
file = files[5]
#df1 = pd.read_excel(in_path_apr_2016_report + file)
df1 = db.download_file(in_path_apr_2016_report,file)
#df2 = pd.read_excel(in_path_apr_2016_report + file, skiprows=1)
df2 = db.download_file(in_path_apr_2016_report,file,skip=1)
## Making Sure Every File contains date the file was created and the foia that created it
#col_list = df.columns.tolist()
#Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
#col_list = [x for x in col_list if isinstance(x, datetime.datetime)==False]
#FOIA_Request = [x for x in col_list if 'FOIA' in x][0]
df1.dropna(how='all', inplace=True)
df1 = df1[["Badge #","Last Name","First Name","Title"]]
df1.dropna(how='all', inplace=True)
df2 = df2[["Name","Position","Hire Date","Hire Date","Employment","Date"]]
df2.dropna(subset=['Name','Position'], inplace=True)
## Drop all null columns
df1 = null_dropper(df1)
df2 = null_dropper(df2)

df1.columns = ["CRID","Investigator_Last_Name","Investigator_First_name","Investigator_Rank"]
df2.columns = ["Investigator_Name","Investigator_Rank","Investigator_City_Hire_Date",
              "Investigator_IPRA_Hire_Date","Investigator_Employment_Status","Investigator_Left_Date"]

##df["FOIA_Request_Number"]=FOIA_Request
#try:
#    df["Report_Produced_Date"]=Report_Produced_Date.date()
#except:
#    df["Report_Produced_Date"]=''
final_df1 = df1
final_df1.reset_index(drop=True,inplace=True)
final_df2 = df2
final_df2.reset_index(drop=True,inplace=True)

metadata_df1 = metadata_dataset(final_df1,file)
metadata_df1.reset_index(drop=True,inplace=True)

metadata_df2 = metadata_dataset(final_df2,file)
metadata_df2.reset_index(drop=True,inplace=True)

In [14]:
saving_file = "IPRA_Investigators_Officers"
final_df1.to_csv(out_path_apr_2016_report+saving_file+".csv",index=False)
final_df1.to_excel(out_path_apr_2016_report+saving_file+".xlsx",index=False)

metadata_df1.to_csv(out_path_apr_2016_report+saving_file+"_metadata.csv",index=False)

saving_file = "IPRA_Investigators_Staff"
final_df2.to_csv(out_path_apr_2016_report+saving_file+".csv",index=False)
final_df2.to_excel(out_path_apr_2016_report+saving_file+".xlsx",index=False)

metadata_df2.to_csv(out_path_apr_2016_report+saving_file+"_metadata.csv",index=False)

### Report 3

In [16]:
file = files[6]
##df = pd.read_excel(in_path_apr_2016_report + file,nrows=20)
df = db.download_file(in_path_apr_2016_report,file,rows=20)
print(df.head())
## Making Sure Every File contains date the file was created and the foia that created it
#col_list = df.columns.tolist()
#Report_Produced_Date = [x for x in col_list if isinstance(x, datetime.datetime)]
#col_list = [x for x in col_list if isinstance(x, datetime.datetime)==False]
#FOIA_Request = [x for x in col_list if 'FOIA' in x][0]

df.dropna(how='all', inplace=True)
## Drop all null columns
df = null_dropper(df)
df.columns = ["CRID","Unit","Police_Shooting","Incident_Date","Address","Direction","Street","City",
              "Party","Initial_Category_Code","Initial_Category_Description","Initital_Category_Type_Code",
             "Initital_Category_Type_Description","Current_Category_Code","Current_Category_Type_Code",
              "Zip","Current_Category_Description","Current_Category_Type_Description","Victim_Age","Victim_Sex",
             "Victim_Race","Complaint_Date","CR_Required"]

## Covert Address Columns into single column
df["Address"] = df['Address'].astype(str)
df["Direction"] = df['Direction'].astype(str)
df["Street"] = df["Street"].astype(str)
df["Address"] = df["Address"].str.replace("nan","")
df["Address"] = df["Address"].str.replace(".0","")
df["Direction"] = df['Direction'].str.replace("nan","")
df["Street"] = df["Street"].str.replace("nan","")

df["Address_of_Incident"] = df[["Address","Direction","Street"]].apply(lambda x: ' '.join(x), axis=1)

df = df[["CRID","Unit","Police_Shooting","Incident_Date","Address","Direction","Street","Address_of_Incident",
         "City","Zip","Party","Initial_Category_Code","Initial_Category_Description","Initital_Category_Type_Code",
         "Initital_Category_Type_Description","Current_Category_Code","Current_Category_Description",
         "Current_Category_Type_Code","Current_Category_Type_Description","Victim_Age","Victim_Sex",
         "Victim_Race","Complaint_Date","CR_Required"]]

#df["FOIA_Request_Number"]=FOIA_Request
try:
    df["Report_Produced_Date"]=Report_Produced_Date.date()
except:
    df["Report_Produced_Date"]=''
final_df = df
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_dataset(final_df,file)
metadata_df.reset_index(drop=True,inplace=True)

      LOG_NO  LastOfOCCURANCE_UNIT_NO POLICE_SHOOTING LastOfFROM_DATETIME  \
0  1000341.0                     15.0             Yes 2006-10-09 17:40:00   
1  1000343.0                      4.0             Yes 2006-10-09 23:09:00   
2  1000343.0                      4.0             Yes 2006-10-09 23:09:00   
3  1000647.0                      9.0             Yes 2006-10-21 01:50:00   
4  1000647.0                      9.0             Yes 2006-10-21 01:50:00   

   LastOfSTREET_NO LastOfSTREET_DIRECTION LastOfSTREET_NME LastOfCITY  \
0            932.0                  North      LATROBE AVE    CHICAGO   
1           8132.0                  South       COLFAX AVE    CHICAGO   
2           8132.0                  South       COLFAX AVE    CHICAGO   
3           4505.0                  South      ASHLAND AVE    CHICAGO   
4           4505.0                  South      ASHLAND AVE    CHICAGO   

  PARTY_TYPE INITIAL_CATEGORY_CODE     ...     CURRENT_CATEGORY_TYPE_CODE  \
0     Victim         

In [17]:
saving_file = "Shootings_Victim_Data"
final_df.to_csv(out_path_apr_2016_report+saving_file+".csv",index=False)
final_df.to_excel(out_path_apr_2016_report+saving_file+".xlsx",index=False)

metadata_df.to_csv(out_path_apr_2016_report+saving_file+"_metadata.csv",index=False)
